# Redes Generativas Adversarias
Alrededor de la mitad del año 2014 algunos de los investigadores asociados al departamento de informática de la Universidad de Montreal proponen un nuevo marco de referencia para la estimación de un modelo Generativo basado en redes neuronales, el cual presentarón en el artículo que posteriormente otorgaría el nombre de este marco de referencia hasta el día de hoy llamado [Generative Adversarial Network](https://arxiv.org/pdf/1406.2661.pdf) *Goodfellow et al* o GAN, por sus siglas en inglés, como se les conoce popularmente. Pero antes que nada se hace necesario responder si quiera brevemente las siguientes cuestiones ¿Qué son los modelos generativos? ¿Qué son los procesos adversarios?

De acuerdo con [developers google](https://developers.google.com/) un [modelo generativo](https://developers.google.com/machine-learning/gan/generative) tiene la capacidad de capturar la probabilidad conjunta P(X,Y) o la densidad de probabilidad P(X) de un grupo de datos, lo que le permite en algunos casos crear objetos similares a dicho conjunto. Por otro lado, un proceso adversario en su forma mas simple es el procedimiento para entrenar dos modelos, para el caso específico de las GAN's son llamados `G` y `D`, a través de un proceso de optimización de juego **minimax**, donde después de un tiempo el entrenamiento se tiende a estabilizar causando que ambos modelos no aprendan mas el uno del otro *Goodfellow et al* .

El marco de referencia propuesto por *Goodfellow et al* consiste en dos redes neuronales: `G` como se le llama al modelo Generativo y `D` que se refiere a un modelo de clasificación que tratará de identificar que imagnes provienen del conjunto de datos real y cuales vienen de `G`. A este clasificador se le es llamado Modelo Discriminativo *Goodfellow et al*. Posteriormente de definir la arquitectura de ambos, se opta por entrenarlos con dos objetivos contrarios: Por parte de `G` se tratará de **maximizar** la probabilidad de que `D` se equivoque. Por otro lado, `D` buscará **minimizar** la probabilidad de caer en dicho error. Después de un amplio entrenamiento proceso de aprendizaje se estabilizará. De acuerdo con las estimaciones de los autores el **punto** donde no existes mas aprendizaje es cuando `D` se acercará a un valor constante de $\frac{1}{2}$. Se propone usar modelos de redes neuronales debido a el amplio y fácil uso del método de optimización basado en backpropagation.

De lo descrito por *Goodfellow et al* en su [artículo](https://arxiv.org/pdf/1406.2661.pdf) me gustaría extraer un fragmento con el que trata de explicar de manera ilustrativa el proceso adversario que existen en este framework que proponen: "El modelo generativo puede ser pensado de manera análoga como un equipo de falsificadores tratando de producir modena falsa los cuales la usan sin detección, por otro lado el modelo discriminativo es análogo a la policía que trata de detectar las falsificaciones. La competencia en este juego lleva a ambos equipos a mejorar en su tarea hasta que las falsificaciones de dinero son indetectables de los genuinos" 

## Detalles del entrenamiento
Este proceso inicia con la creación de un vector de ruido denotado por `z` que en algunas ocasiones es llamado *latent vector*. La creación de este vector obedece a una distribución de porbabilidad $p_z(z)$, donde en la mayoría de los casos es una distribución uniforme o Normal. 

Después se representa un mapeo al espacio de datos con $G(z; \theta_g)$, donde la función `G` es diferenciable y es una red neuronal cuyos pesos pertenecen al conjunto $\theta_g$. En paralelo también se define una función `D` diferenciable en forma de una red neuronal, cuyo mapeo de entrada es definido por $D(x; \theta_d)$ donde su salida es un escalar que **refleja** la probabilidad de que `x` provenga del conjunto de datos reales en lugar de $p_g$ (distribución de los datos generados). El modelo final puede ser visto de la siguiente manera:

![Diagrama DCGAN cortesía de developers google](https://developers.google.com/machine-learning/gan/images/gan_diagram.svg)

Los autores defienen los siguientes creterios de optimización y suposiciones: 
* **Criterio de optimización**: Entrenar `D` para maximizar la probabilidad de que los valores que provengan del dataset sean asignados con la etiqueta binaria 1, mientras que los que provengan de la distribución $p_g$ sean asignados con 0. 
* **Criterio de optimización***: Se entrenará `G` para que `D` asigne $p_g$ como 1.
* **Suposición**: El aprendizaje del generardor podrá ser efectivamente direccionado si el discriminador del que aprende se encuentra en su optimo global, es decir:$$D_G^*(x) = \frac{p_{data}(x)}{p_{data}(x) + p_G(x)}$$ Lo que implica que en ocaciones se deba enternar el Discriminador múltiples veces antes de entrenar el generador.

De acuerdo con *Goodfellow et all* la forma matemática de este problema es la siguiente:

$$
min_G~max_D~V(D, G) = \mathbf{E}_{x \sim p_{data}(x)}(logD(x)) + \mathbf{E}_{z \sim p_{z}(z)}(log(1 - D(G(z)))) \tag{1}
$$

Para un mejor entendimiento, recorramos la ecuación parte por parte:
* Lo referente a $\mathbf{E}_{x \sim p_{data}(x)}(...)$ tiene varias partes: $\mathbf{E}$ es el valor esperado o promedio y $x \sim p_{data}(x)$ especifíca el conjunto al que se va a evaluar su promedio, que para este caso vienen siendo todos los datos provenientes de los originales.
* Con respecto a $\mathbf{E}_{z \sim p_{z}(z)}(...)$ Las partes funcionan de igual manera que en el anterior, un único distinto es que este apartado $z \sim p_{z}(z)$ especifica que el conjunto obetivo es `z` en este caso.

La figura 1 de *Goodfellow et al* ilustra de gran manera este proceso:
![Proceso de entrenamiento de GAN obtenido de Goodfellow et al](../imagenes/figura_1_Goodfellow_GAN.PNG)

Explicando el código de colores y líneas: 
1. La línea punteada azul corresponde al valor escalar de `D` en el espacio de datos.
2. La línea de puntos negra corresponde al valor de la función de densidad de probabilidad de los datos reales en el dominio definido en la gráfica. **Esta es la distribución que queremos imitar con nuestro modelo generativo**.
3. La linea continua verde correponde la función de densidad probabilidad de las imágenes generadas en ese instante de tiempo por `G`.
4. La línea horizontal `x` es el espacio de datos completo.
5. Las líneas continuas negras que van de la línea horizontal marcada como`z` a la líena horizontal `x` corresponde al mapeo realizado por $G(z)$ en el espacio de datos.

En el instante de tiempo $(a)$ podemos asumir que las dos distribuciones (las generadas y las originales) son cercanas y `D` es un clasificador acercandose a su punto óptimo.

En el instante de tiempo $(b)$ el discriminador `D` llega a su punto optimo, y por medio del aprendizaje de G este se acerca aún mas a la distribución deseada.

En el instante de tiempo $(c)$ el gradiente porvisto por el discriminador `D` va guiando de mejor manera los pesos de `G` para que este pueda mapear eficazmente el *latent vector* con el espacio de datos donde es mas probable encontrar los datos originales.

Por último, en el instante de tiempo $(d)$ el discriminador no puede direccionar mas el gradiente de `G` ya que las dos distribuciones son iguales, lo que hace `D` no pueda identificar la diferencia. Así su valor a partir de este punto siempre será $\frac{1}{2}$

Como vimos en la figura de propositos pedagógicos anterior, `D` es una parte crucial del problema debido a que debe porder diferenciar complemetamente entre los datos reales y los generados, ya que es computacionalmente prohibitivo. Se trará de que `D` sea completamente entrenado antes de que lo haga `G`, para esto se crea un nuevo hiperparámetro denotado como $k$ el cual es la cantidad de pasos de optimización que el algoritmo hace sobre `D` antes de hacer un paso en `G`. El buen ajuste de $k$ causará que `D` se aproxime al punto óptimo y hace que `G` cambie lo suficientemente lento. *Goodfellow et al* presentan el algoritmo en la siguiente imagen.

![Algoritmo de entrenamiento GAN obtenido de Goodfellow et al](../imagenes/Algoritmo_entrenamiento_GAN.PNG)

De acuerdo con *Goodfellow et al* la ecuación $(1)$ en ocaciones no provee suficiente gradiente para que `G` aprende, esto sobre todo en el principio del entrenamiento. Según los autores `D` puede rechazar los ejemplos que tengan alta confiabilidad de resultado, es decir, al principio para `D` es muy claro que los ejemplo generados con muy disntintos a los reales. **Así que en lugar de netrenar a `G` para minimizar $log(1- D(G(z)))$ trate de maximizar $logD(G(z))$**. Esto eventualmente nos llevará al mismo punto de entrenamiento sin embargo `D` proverá un mejor gradiente para `G`.

A continuación, tataremos de imitar de la mejor manera la arquitectura para una GAN usando los conjuntos de datos MNIST y CIFAR-10:

# Configurando nuestro entorno
Lo primero que necesitamos es llamar las librerías, funciones y conjuntos de datos necesarios para realizar la tarea:

In [ ]:
import os
import numpy as np
from math import sqrt
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython import display
from tqdm.notebook import tqdm
import tensorflow_datasets as tfds
import tensorflow as tf
from helper_func.custom_layers import Maxout
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.train import Checkpoint, CheckpointManager

plt.rcParams['figure.figsize'] = [12, 8]

In [ ]:
SEED = 777
M = 50000
LATENT_SIZE = 100
LEARNING_RATE = .1
BATCH_SIZE = 100
BUFFER = 1000
INIT_MOMENTUM = .5
EPOCHS = 231
K = 1

Cargamos los datos, para esto vamos a usar el conjunto de números escritos a mano, popularmente llamado [MNIST](http://yann.lecun.com/exdb/mnist/).

In [ ]:
mnist, mnist_test = tfds.load('mnist', split=['train[:50000]', 'train[50000:]'], data_dir='./', as_supervised=True)

Observemos los primeros 20 elementos de nuestro dataset

In [ ]:
sample = mnist.take(20)

fig = plt.gcf()

for i, x in enumerate(tfds.as_numpy(sample)):
    image, _ = x
    ax = fig.add_subplot(4, 5, i+1)
    ax.imshow(image[:, :, 0], cmap='gray')
    ax.axis('off')

# Adecuando el conjunto de datos
En este apartado adecuaremos el dataset para que sea entrada del Discriminator. También debemos hallar el valor inicial del bias de la última capa del generador como lo hacen los autores.

In [ ]:
def norm_img(image, label):
    return tf.keras.backend.flatten(tf.cast(image, tf.float32)/255.)

In [ ]:
mnist = mnist.map(
    norm_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
mean_dataset = mnist.reduce(np.float32(0), lambda x,y: x/float(M) + y).numpy()
mean_dataset = np.clip(mean_dataset, 1e-7, 1-1e-7)
init_last_bias_gen = tf.convert_to_tensor(np.log(mean_dataset / (1. - mean_dataset)), dtype= tf.float32)

In [ ]:
mnist_test = mnist_test.map(norm_img, num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(3000).batch(1000)

In [ ]:
mnist = mnist.shuffle(BUFFER)
mnist = mnist.batch(BATCH_SIZE)
mnist = mnist.cache()
mnist = mnist.repeat()
mnist = mnist.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
mnist = iter(mnist)


# Construcción de modelos
Construimos nuestro **Generator** y **Discriminator** la GAN basada en en conjunto de datos MNIST. La arquitectura desarrollada por el investigador la puede encontrar [aqui](https://github.com/goodfeli/adversarial/blob/master/mnist.yaml):

## Generator
Como se dijo el **Generator** es un modelo basado en redes neuronales encargado de aprender la distribución del conjunto de datos real a partir del proceso adversario. Por otra parte, su entrada corresponde a un "latent vector" $z$ el cual tiene una distriución de probabilidad $p_z$ asociada. En este caso el autor elige en vector $z \in \mathbb{R}^{100}$ y $p_z = U(-\sqrt{3}, \sqrt{3})$

In [ ]:
def make_generator():
    # Los autores hacen una serie de inicializaciones de pesos que se replicarán
    init_kernel_gen = tf.random_uniform_initializer(-.05 , .05, seed=SEED)
    model = Sequential([
        Dense(1200, activation='relu', kernel_initializer=init_kernel_gen, input_shape=(100,)),
        Dense(1200, activation='relu', kernel_initializer=init_kernel_gen),
        Dense(784, activation='sigmoid', kernel_initializer=init_kernel_gen)
    ])
    
    return model  

In [ ]:
gen = make_generator()
gen.summary()

Aplicamos la inicialización del bias de la última capa del **Generator**

In [ ]:
_ = gen.trainable_variables[-1].assign(init_last_bias_gen)

In [ ]:
gen = make_generator()
noise = tf.random.uniform([1, 100], minval=-1.*sqrt(3.), maxval=sqrt(3.), seed=SEED)
generated_image = gen(noise, training=False)
generated_image = tf.reshape(generated_image, (28, 28))
plt.imshow(generated_image.numpy(), cmap='gray')
_ = plt.axis('off')

## Discriminator
El **discriminator**, al igual que generator es un modelo basado en redes neuronales. Su función es definir si una entrada es similar o no al conjunto real. Los autores implementan la capa [Maxout](https://arxiv.org/pdf/1302.4389.pdf) propuesta por *Goodfellow et al*, la cual (en una descripción breve) obtiene el máximo de una serie de combinaciones lineales de las entradas y es que para ellos explota de mejor manera las características de la técnica *Dropout*, para mayor información remitace al artículo en el link.

In [ ]:
def make_discriminator():
    # El autor hace una serie de inicializaciones de pesos que se replicarán
    init_kernel = tf.random_uniform_initializer(-.005 , .005, seed=SEED)
    model = Sequential([
        Maxout(240, 5, kernel_initializer=init_kernel, input_shape=(784,)),
        Dropout(.2, seed=SEED),
        Maxout(240, 5, kernel_initializer=init_kernel),
        Dropout(.5, seed=SEED),
        Dense(1, activation='sigmoid', kernel_initializer=init_kernel),
    ])
    
    return model

In [ ]:
disc = make_discriminator()
disc.summary()

In [ ]:
disc = make_discriminator()
decision = disc(tf.reshape(generated_image, (1, -1)))
print(decision)

## Definición de la métrica de evaluación
Siempre es bueno tener alguna métrica que nos pueda direccionar nuestro entrenamiento. En este caso ya que trabajamos con un problema no supervisado, es difícil encontrar alguna, sin embargo los autores eligen el la métrica **log likelihood** a través de una ventana Gaussiana de Parzen, esto con el fin de estimar la probabilidad que los datos de testeo se encuentren en la distribución $p_g$, para mayor información buscar leer el siguiente [artículo](http://proceedings.mlr.press/v9/desjardins10a/desjardins10a.pdf). Debido a que el calculo es muy intensivo nos limitaremos a probar 1000 muestras generadas. Para el caso del Dataset MNIST los autores definen $\sigma = 0.2$

In [ ]:
def parzen(data, samples, sigma):
    x = data[:, None, :]
    s = samples[None, :, :]
    fracc = -.5 * ((x - s)/sigma)**2
    exp_gaus = tf.reduce_sum(fracc, axis=-1)
    
    max_ = tf.reduce_max(exp_gaus, axis = -1)
    
    E = max_ + tf.math.log(tf.reduce_mean(tf.math.exp(exp_gaus - max_[:, None]), axis=-1))
    Z = samples.shape[1] * tf.math.log(sigma * tf.math.sqrt(np.pi * 2))
    
    return tf.reduce_mean(E - Z)

In [ ]:
def ll_metric(test):
    noise = tf.random.uniform((200, 100), minval=-1.*sqrt(3.), maxval=sqrt(3.), seed=SEED)
    S = gen(noise, training=False)
    ll = []
    for test_batch in tqdm(mnist_test, desc='Log-likelihood Valid', leave=False):
        ll.append(parzen(test_batch, S, .2).numpy())
    ll = np.array(ll)
    return ll.mean(), ll.std()

In [ ]:
ll_metric(mnist_test)

## Definición funciones de costo y optimizadores
En este apartado se definirán las funciones de costo para la actualización de las variables del **Discriminator** y la función del costos del **Generator**. Además se especificará los optimizadores de para el **Generator** y **Discriminator** con la respectiva programación de la tasa de aprendizaje y momentum. Empezaremos por estos últimos:   

In [ ]:
# Forma de disminución de tasa de aprendizaje usada por los autores
class ExponentialDecayGANPaper(LearningRateSchedule):
    
    def __init__(self, init_lr=.1, decay_factor = 1.0003, min_lr=1e-6):
        self.init_lr = tf.cast(init_lr, tf.float32)
        self.decay_factor = tf.cast(decay_factor, tf.float32)
        self.min_lr = tf.cast(min_lr, tf.float32)
    
    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        lr = self.init_lr * tf.math.pow(tf.constant(1., dtype=tf.float32)/self.decay_factor, step)
        return tf.cond(lr > self.min_lr, lambda: lr, lambda: self.min_lr)
    
    def get_config(self):
        return {
            'init_lr': self.init_lr,
            'decay_factor': self.decay_factor,
            'min_lr': self.min_lr,
        }        

In [ ]:
lr_scheduler = ExponentialDecayGANPaper(init_lr=LEARNING_RATE, decay_factor=1.000004)

In [ ]:
class MomentumAdjustor():
    def __init__(self, init_momentum=.5, final_momentum=.7, start=1, saturate=250):
        self.init_momentum=init_momentum
        self.final_momentum=final_momentum
        self.start=start
        self.saturate=saturate
        
    def __call__(self, epoch):
        eta = float(epoch - self.start) / float(self.saturate - self.start)
        if eta < 0.:
            eta = 0.
        elif eta > 1.:
            eta = 1.
        return np.float32(self.init_momentum*(1. - eta) + eta*self.final_momentum)

In [ ]:
gen_opt = SGD(learning_rate=lr_scheduler, momentum=INIT_MOMENTUM)
disc_opt = SGD(learning_rate=lr_scheduler, momentum=INIT_MOMENTUM)

In [ ]:
momentum_scheduler = MomentumAdjustor(INIT_MOMENTUM, final_momentum=.7, start=1, saturate=250)

In [ ]:
loss = BinaryCrossentropy()

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = loss(tf.ones_like(real_output), real_output)
    fake_loss = loss(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return .5 * total_loss

In [ ]:
def generator_loss(fake_output):
    return loss(tf.ones_like(fake_output), fake_output)

## Puntos de control
Acá ubicaremos donde serán gardados los puntos de control del epoch mas actual y del epoch con la mejr métrica de entrenamiento.

In [ ]:
checkpoint = Checkpoint(gen=gen, disc=disc, gen_opt=gen_opt, disc_opt=disc_opt)
manager_best_metric = CheckpointManager(checkpoint, directory="./checkpoint_mnist_model/best_metric", max_to_keep=1)
manager_cpkt = CheckpointManager(checkpoint, directory="./checkpoint_mnist_model/train_progress", max_to_keep=5)

## Creando un ciclo de entrenamiento
A continuación crearemos un ciclo de entrenamiento dependiedo de las ocaciones que el **Discriminator** se debe actualizar ($k$) con respecto al **Generator**, recuerde que entre mayor cantidada de actualizaciones requiera `D` mas lento será el proceso de entrenamiento. en este caso se definío como $k = 1$. También haremos una función para la visualización del entrenamiento de manera cualitativa:

In [ ]:
def generate_image(generator, row, col, seed = SEED):
    latent_vector = tf.random.uniform((int(row*col),LATENT_SIZE), minval=-1.*sqrt(3.), maxval=sqrt(3.), seed=seed)
    gen_images = generator(latent_vector, training=False)
    
    fig = plt.gcf()
    
    for i in range(int(row*col)):
        ax = fig.add_subplot(int(row), int(col), i+1)
        ax.imshow(tf.reshape(gen_images[i], (28,28)), cmap='gray')
        ax.axis('off')
    plt.show()

In [ ]:
def train_disc_only(images):
    noise = tf.random.uniform((images.shape[0],LATENT_SIZE), minval=-1.*sqrt(3.), maxval=sqrt(3.), seed=SEED)   
        
    with tf.GradientTape() as tape:
        generate_imgs = gen(noise, training=True)
        
        real_output = disc(images, training=True)
        fake_output = disc(generate_imgs, training=True)
        
        disc_loss = discriminator_loss(real_output, fake_output)
        
    grads = tape.gradient(disc_loss, disc.trainable_variables)
    disc_opt.apply_gradients(zip(grads, disc.trainable_variables))

In [ ]:
def train_gen_step():
    noise = tf.random.uniform((BATCH_SIZE,LATENT_SIZE), minval=-1.*sqrt(3.), maxval=sqrt(3.), seed=SEED)   

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generate_imgs = gen(noise, training=True)
        
        fake_output = disc(generate_imgs, training=True)
        
        gen_loss = generator_loss(fake_output)

        
    gen_grads = gen_tape.gradient(gen_loss, gen.trainable_variables)
    disc_grads = disc_tape.gradient(disc_loss, disc.trainable_variables)

### Entrenamiento
Ahora realizaremos el entrenamiento. Tenga encuenta que en un Computador sin GPU cada epoch tomará entre 2 y 3 minutos. Puede evitar el entrenamiento cargando el punto de control en el epoch 50 

####  Cargar punto de control epoch 50

In [ ]:
manager.restore_or_initialize()

#### Loop de entrenamiento

In [ ]:
max_ll = -np.inf

In [ ]:
for epoch in range(EPOCHS):
    gen_opt.momentum.assign(momentum_scheduler(epoch+1))
    disc_opt.momentum.assign(momentum_scheduler(epoch+1))
    for step in tqdm(range(M // BATCH_SIZE), desc='Steps', leave=False):
        
        for _ in range(K):
            images = mnist.get_next_as_optional()
            images = images.get_value()
            train_disc_step(images)
        
        train_gen_step()
    
    ll_mean, ll_std = ll_metric(mnist_test)
    if max_ll < ll_mean:
        manager_best_metric.save(checkpoint_number=epoch+1, check_interval=False)
        max_ll = ll_mean
    manager_cpkt.save(checkpoint_number=epoch+1)
    display.clear_output(wait=True)
    print(f'Epoch: {epoch + 1}')
    print(f'Log-likelihood: {ll_mean:4.2f} +/- {ll_std:4.4f}, Max ll: {max_ll:4.2f}')
    generate_image(gen, 4, 5)

## Verificar resultados
A continuación de manera cualitativa verificaremos si los datos generados son parecidos a los reales. Los autores usan varias métricas para esto, si tiene dudas remítase al [artículo](https://arxiv.org/pdf/1406.2661.pdf)

In [ ]:
generate_image(gen, 4, 5)

In [ ]:
fig = plt.gcf()
real_images = mnist.get_next_as_optional()
real_images = real_images.get_value()
real_images = real_images[:20]
for i in range(20):
    ax = fig.add_subplot(4, 5, i+1)
    ax.imshow(tf.reshape(real_images[i,:], (28,28)), cmap='gray')
    ax.axis('off')
plt.show()

In [ ]:
# uppper_limit_noise = tf.random.uniform((LATENT_SIZE,), minval=-1.*sqrt(3.), maxval=sqrt(3.))
# bottom_limit_noise = tf.random.uniform((LATENT_SIZE,), minval=-1.*sqrt(3.), maxval=sqrt(3.))

uppper_limit_noise = sqrt(3.) * tf.ones((LATENT_SIZE,))
bottom_limit_noise = -uppper_limit_noise

generate_interp_noise = tf.linspace(bottom_limit_noise, uppper_limit_noise, 20, axis=0)

In [ ]:
fig = plt.figure(figsize=(16,2))
generate_interp_imgs = gen(generate_interp_noise, training=False)
for i in range(20):
    ax = fig.add_subplot(1, 20, i+1)
    ax.imshow(tf.reshape(generate_interp_imgs[i], (28,28)), cmap='gray')
    ax.axis('off')

In [ ]:
disc(tf.ones((1,784)))

In [ ]:
disc.save('disc.h5')

In [ ]:
saved_dir = './discriminator_mnist/1/'
tf.saved_model.save(disc, saved_dir)